In [7]:
import cv2
import numpy as np
    


In [8]:
def CreateUI():
	UIHeight = 300
	UIWidth = 800

	UIBG = np.zeros((UIHeight,UIWidth,3),dtype=np.uint8)

	BoxHeight = 200
	BoxWidth = 200
	Gap = 25

	YStart = int((UIHeight - BoxHeight)/2)

	for i in range(3):
		XStart = Gap + i*(BoxWidth+Gap)

		BoxTopLeft = (XStart,YStart)
		BoxBottomRight = (XStart+BoxWidth , YStart+BoxHeight)

		cv2.rectangle(UIBG,BoxTopLeft,BoxBottomRight,(192,242,30),-1)
	return UIBG

UIBG = CreateUI()
MarkerIDs = {0,1,2,3}

Calibrated = False
VirtualScreen = None
MarkerCorners = {}
aruco_dict = cv2.aruco.getPredefinedDictionary(cv2.aruco.DICT_4X4_50)
aruco_params = cv2.aruco.DetectorParameters()
cam = cv2.VideoCapture(0,cv2.CAP_V4L2)
if not cam.isOpened():
	print("Cannot Open Camera")
	exit()

In [9]:
while True:
	success,frame = cam.read()
	if not success:
		print("No Frame")
		break
	gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
	corners,ids,rejected = cv2.aruco.detectMarkers(gray,aruco_dict,parameters=aruco_params)

	key = cv2.waitKey(1) & 0xFF
	if key == ord('c') and ids is not None and len(ids) == 4:
		TempCorners = {}
		MarkerCenters = []

		for i,MarkerID in enumerate(ids):
			if MarkerID[0] in MarkerIDs:
				TempCorners[MarkerID[0]] = corners[i][0]
				
				Center = np.mean(corners[i][0] , axis=0)
				MarkerCenters.append((int(Center[0]) , int(Center[1])))

		if len(TempCorners) == 4:
			MarkerCorners = TempCorners
			MarkerCenters.sort(key=lambda p: p[1])

			TopPoints = sorted(MarkerCenters[:2], key=lambda p:p[0])
			BottomPoints = sorted(MarkerCenters[2:], key=lambda p:p[0])

			TopLeft = TopPoints[0]
			TopRight = TopPoints[1]
			BottomLeft = BottomPoints[0]
			BottomRight = BottomPoints[1]

			VirtualScreen = np.array([TopLeft,TopRight,BottomRight,BottomLeft], dtype=np.float32)

			Calibrated = True
			print("Calibration Done")

	if Calibrated and ids is not None:
		cv2.aruco.drawDetectedMarkers(frame,corners,ids)
		PerfectPoints = []
		CurrentPoints = []
		for i,MarkerID in enumerate(ids):
			if MarkerID[0] in MarkerIDs:
				for point in MarkerCorners[MarkerID[0]]:
					PerfectPoints.append(point)

				for point in corners[i][0]:
					CurrentPoints.append(point)

		if len(PerfectPoints) >= 4:
			PerfectPoints = np.array(PerfectPoints, dtype=np.float32)
			CurrentPoints = np.array(CurrentPoints, dtype=np.float32)

			Matrix,_ = cv2.findHomography(PerfectPoints,CurrentPoints)

			if Matrix is not None:
				CurrentVirtualScreen = cv2.perspectiveTransform(VirtualScreen.reshape(-1,1,2), Matrix)

				h,w,c = UIBG.shape
				SourcePoints = np.array([[0,0],[w,0],[w,h],[0,h]] , dtype=np.float32)

				WarpMatrix,_ = cv2.findHomography(SourcePoints,CurrentVirtualScreen)
				if WarpMatrix is not None:
					WarpedUI = cv2.warpPerspective(UIBG, WarpMatrix, (frame.shape[1],frame.shape[0]))
					mask = np.sum(WarpedUI,axis=2) > 0
					frame[mask] = WarpedUI[mask]

	if not Calibrated:
		cv2.putText(frame, "Show all markers and press 'c'!",(50,50),cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,255),2)

	cv2.imshow("VideoFeed",frame)

	if key == ord('q'):
		break

cam.release()
cv2.destroyAllWindows()

Calibration Done
